# GRASP

**GRASP** (General Responsibility Assignment Software Patterns) - 9 zasad odpowiedzialności w projektowaniu obiektowym (Craig Larman, 1997).

Cel: **łatwa rozbudowa, łatwe utrzymanie, minimalizacja wpływu zmian.**

1. controller
2. creator
3. indirection
4. information expert
5. low coupling
6. high cohesion
7. polymorphism
8. protected variations
9. pure fabrication

## Low Coupling & High Cohesion

<div style="border: solid 1px;padding: 20px;text-align: center">
Dobry kod charakteryzuje się <b>wysoką spójnością</b> (<i>ang. high cohesion</i>) i <b>luźnymi sprzężeniami</b> (<i>ang. loose coupling</i>).
</div>

## Low Coupling (Luźne sprzężenia)

**Coupling** - stopień zależności jednej klasy od drugiej.

### Problem: Silne sprzężenie

In [ ]:
class User:
    def __init__(self, name):
        self.name = name
    
    def say_hello(self):
        print(f"Cześć, nazywam się {self.name}")


class Main:
    def __init__(self):
        user = User("Mark")
        user.say_hello()

`Main` jest **powiązana** (zależy od) `User`. Zmiana w `User` może zepsuć `Main`:

In [ ]:
# Zmiana: dodanie parametru age
class User:
    def __init__(self, name, age):  # <- Nowy parametr
        self.name = name
        self.age = age
    
    def say_hello(self):
        print(f"Cześć, nazywam się {self.name}")


# Main przestaje działać!
class Main:
    def __init__(self):
        user = User("Mark")  # <- TypeError: missing 1 required positional argument: 'age'
        user.say_hello()

**Efekt kaskadowy**: Zmiana w `User` wymusza zmianę w `Main` (i wszystkich innych klasach używających `User`).

W dużych projektach (dziesiątki/setki klas) = **trudne utrzymanie i rozbudowa**.

---

**Analogia**: Koło w samochodzie.
- Luźne sprzężenie: Wymieniasz tylko koło, nie silnik.
- Silne sprzężenie: Wymiana koła wymaga wymiany silnika i układu kierowniczego.

## Przykład: Zoo ze zwierzętami

### Źle: Silne sprzężenie

In [ ]:
class Monkey:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Monkey named {self.name}"


class Lion:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Lion named {self.name}"


In [ ]:
class Zoo:
    def __init__(self, monkey, lion):
        self.__monkey = monkey
        self.__lion = lion
    
    def display_zoo_animals(self):
        print(self.__monkey)
        print(self.__lion)


In [ ]:
# client code
monkey = Monkey("Rafiki")
lion = Lion("Simba")
zoo = Zoo(monkey, lion)
zoo.display_zoo_animals()

**Problem**: Dodanie nowego zwierzęcia wymaga edycji `Zoo`:

In [ ]:
class Warthog:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Warthog named {self.name}"


# Musimy zmienić Zoo:
class Zoo:
    def __init__(self, monkey, lion, warthog):  # <- Edycja __init__
        self.__monkey = monkey
        self.__lion = lion
        self.__warthog = warthog  # <- Nowy atrybut
    
    def display_zoo_animals(self):
        print(self.__monkey)
        print(self.__lion)
        print(self.__warthog)  # <- Edycja metody

### Dobrze: Luźne sprzężenie (polimorfizm)

In [ ]:
# TODO: zdefinijuj interfejs Animal

class Monkey(Animal):
    pass


class Lion(Animal):
    pass


class Warthog(Animal):
    pass


In [ ]:
# TODO: Rozluźnij sprzężenia pomiędzy Zoo a klasami Animal


In [ ]:
# client code
# TODO: użyj nowej struktury

**Teraz**: Nowe zwierzę = nowa klasa, **zero zmian w `Zoo`**.

---

## Poziomy sprzężeń (od najluźniejszego do najsilniejszego)

1. **Dependency (USES-A)** - klasa używa innej, ale nie trzyma referencji
   ```python
   class A:
       def method(self, b: B):  # <- B tylko jako parametr
           b.do_something()
   ```

2. **Association (HAS-A)** - klasa trzyma referencję
   ```python
   class A:
       def __init__(self, b: B):
           self.b = b  # <- Referencja do B
   ```

3. **Composition (OWNS-A)** - klasa tworzy i posiada instancję
   ```python
   class A:
       def __init__(self):
           self.b = B()  # <- A tworzy B
   ```

4. **Inheritance (IS-A)** - najsilniejsze sprzężenie
   ```python
   class A(B):  # <- A dziedziczy po B
       pass
   ```

**Zasada**: Projektuj klasy niezależne. Jeśli dziedziczysz, rób to po interfejsach/klasach abstrakcyjnych (patrz: DIP).

## High Cohesion (Wysoka spójność)

**Cohesion** - miara tego, jak **skupione** są odpowiedzialności klasy.

Klasa powinna:
- Być prosta do zrozumienia
- Mieć niewiele metod
- Współpracować z innymi klasami przy złożonych zadaniach

**Jeśli masz wątpliwości** - oddeleguj metodę do innej klasy.

### Przykład: Niska spójność

In [ ]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    # Odpowiedzialność 1: Zarządzanie danymi użytkownika
    def get_name(self):
        return self.name
    
    # Odpowiedzialność 2: Walidacja email
    def validate_email(self):
        return '@' in self.email
    
    # Odpowiedzialność 3: Wysyłanie email
    def send_email(self, message):
        print(f"Wysyłam email do {self.email}: {message}")
    
    # Odpowiedzialność 4: Zapis do bazy
    def save_to_database(self):
        print(f"Zapisuję {self.name} do bazy")
    
    # Odpowiedzialność 5: Generowanie raportu
    def generate_report(self):
        return f"Raport dla {self.name}"

**Problem**: `User` robi **wszystko** - niska spójność, trudne testowanie, trudne utrzymanie.

### Przykład: Wysoka spójność

In [ ]:
# Klasa 1: Tylko dane użytkownika
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def get_name(self):
        return self.name

# Klasa 2: Tylko walidacja
class EmailValidator:
    @staticmethod
    def validate(email):
        return '@' in email

# Klasa 3: Tylko wysyłka email
class EmailSender:
    @staticmethod
    def send(email, message):
        print(f"Wysyłam email do {email}: {message}")

# Klasa 4: Tylko zapis
class UserRepository:
    @staticmethod
    def save(user: User):
        print(f"Zapisuję {user.name} do bazy")

# Klasa 5: Tylko raporty
class ReportGenerator:
    @staticmethod
    def generate(user: User):
        return f"Raport dla {user.name}"

# Użycie
user = User("Jan", "jan@example.com")
if EmailValidator.validate(user.email):
    EmailSender.send(user.email, "Witaj!")
    UserRepository.save(user)
    print(ReportGenerator.generate(user))

**Teraz**: Każda klasa ma **jedną odpowiedzialność** - wysoka spójność.

Jak ocenić spójność klasy?

Jednym z wyznaczników spójności klasy jest to, czy jej metody operują na wspólnym zbiorze atrybutów.
Jeżeli tak — oznacza to, że klasa jest spójna. Jeżeli natomiast można wyróżnić, na przykład, dwie grupy metod działające na różnych zestawach atrybutów, stanowi to silną przesłankę do podziału klasy na dwie mniejsze. Taka klasa nie jest spójna — realizuje dwie słabo ze sobą powiązane odpowiedzialności.

## Związek: Low Coupling + High Cohesion

| Aspekt | Low Coupling | High Cohesion |
|--------|--------------|---------------|
| Skupienie | **Między klasami** | **Wewnątrz klasy** |
| Pytanie | Jak bardzo klasy zależą od siebie? | Jak skupiona jest klasa? |
| Cel | Minimalizuj zależności | Maksymalizuj spójność |
| Efekt | Łatwa zmiana (bez efektów kaskadowych) | Łatwe zrozumienie i testowanie |

**Ideał**: Luźne sprzężenia między klasami + wysoka spójność wewnątrz klas.

---

## Podsumowanie

### Low Coupling
- Minimalizuj zależności między klasami
- Używaj interfejsów/klas abstrakcyjnych
- Unikaj dziedziczenia po konkretnych klasach

### High Cohesion
- Jedna klasa = jedna odpowiedzialność (patrz: SRP)
- Deleguj metody do innych klas jeśli wątpliwości
- Klasa powinna być prosta do zrozumienia

**Związek z SOLID**:
- Low Coupling = DIP (zależności od abstrakcji)
- High Cohesion = SRP (jedna odpowiedzialność)